In [4]:
import numpy as np
import csv
import string

x_train = np.genfromtxt('processed_train.txt', delimiter=',')
x_train = np.delete(x_train,-1,axis = 1)
x_test = np.genfromtxt('processed_test.txt', delimiter=',')
x_test = np.delete(x_test,-1,axis = 1)
with open('../data/train_labels.csv') as f:
    reader = csv.reader(f, delimiter=",")
    label_train = list(reader)   
del label_train[0]
label_train = np.asarray(label_train)[:,1]
label_train = label_train.tolist()

import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('../data/train_labels.csv')
obj_df = df.select_dtypes(include=['object']).copy()
obj_df.head()

lb_class = LabelEncoder()
obj_df['Category_code']=lb_class.fit_transform(obj_df['Category'])
obj_df[['Category','Category_code']]
y_train = np.asarray(obj_df)[:,1]
y_train = y_train.astype('int')

In [5]:
def softmax(a):
    result = []
    for i in a:
        result.append(np.exp(i)/sum(np.exp(i)))
    result = np.asarray(result)
    return result

def modelbuiler(n_layer, dim_layer, itera):
    x=[]
    x = np.concatenate((np.zeros((1,n_sample)),x_in.T),axis = 0)
    W = []
    W.append(np.random.randn(dim_layer+1,n_feature+1))
    if n_layer >=3:
        for i in range(1,n_layer-1):
            W.append(np.random.randn(dim_layer+1,dim_layer+1))
    W.append(np.random.randn(n_class,dim_layer+1))
    model = {}
    t = np.zeros((n_class,n_sample))
    for i in range(0,n_sample):
        t[y_in[i],i] = 1
    
    for iteration in range(0,itera):
        #Forward Prop
        a=[]
        z=[]
        a.append(np.dot(W[0],x))
        z.append(np.tanh(a[0]))
        z[0][0]=np.ones_like(z[0][0])
        if n_layer >=3:
            for i in range(1,n_layer-1):
                a.append(np.dot(W[i],z[i-1]))
                z.append(np.tanh(a[i]))
                z[i][0]=np.ones_like(z[i][0])
        a.append(np.dot(W[-1],z[-1]))
        z.append(softmax(a[-1]))        
        
        #Back Prop Gradient Descent
        delta = []
        dW=[]
        h_prime = []
        for i in z:
            h_prime.append(np.ones_like(i)-np.multiply(i,i))
        delta.append(z[-1]-t)
        dW.append(np.dot(delta[0],z[1].T))
        if n_layer >= 3:
            for i in range(1,n_layer-1):
                delta.append(np.multiply(h_prime[-1-i],np.dot(W[-i].T,delta[-1])))
                dW.append(np.dot(delta[-1],z[-i-2].T))
        delta.append(np.multiply(h_prime[0],np.dot(W[1].T,delta[-1])))
        dW.append(np.dot(delta[-1],x.T))
        
        for i in range(0,len(W)):
            W[i]+=-epsilon*dW[-i-1]
        
    model={'W':W}
    return model

#Estimate x
def guess(model, x_guess):
    W = model['W']
    x=[]
    x = np.concatenate((np.zeros((1,n_sample)),x_guess.T),axis = 0)
    a=[]
    z=[]
    a.append(np.dot(W[0],x))
    z.append(np.tanh(a[0]))
    z[0][0]=np.ones_like(z[0][0])
    if layer_number >=3:
        for i in range(1,layer_number-1):
            a.append(np.dot(W[i],z[i-1]))
            z.append(np.tanh(a[i]))
            z[i][0]=np.ones_like(z[i][0])
    a.append(np.dot(W[-1],z[-1]))
    z.append(softmax(a[-1]))
    return z[-1]




In [6]:
x_in = x_train[0:5000]
y_in = y_train[0:5000]
x_try = x_train[5000:10000]
y_try = y_train[5000:10000]

n_sample = x_in.shape[0]
n_feature = x_in.shape[1]
n_class = 31

#Utilize NN Cross Validation
itera = 300
epsilon_valid = [0.01]
layer_number_valid = [3]
layer_dimension_valid = [500]
error = np.zeros((len(epsilon_valid),len(layer_number_valid),len(layer_dimension_valid)))
for i,epsilon in enumerate(epsilon_valid):
    for j,layer_number in enumerate(layer_number_valid):
        for k,layer_dimension in enumerate(layer_dimension_valid):
            model = modelbuiler(layer_number,layer_dimension, itera)
            y_prob = guess(model, x_try)
            result = np.argmax(y_prob,0)
            diff = y_try-result
            diff = diff.astype(bool).astype(int)
            error[i,j,k] = sum(diff)
            print(error)
            
min_error = []
min_error_loc = []
for i in error:
    for j in i:
        min_error.append(min(j))
for i in np.where(error==min(min_error)):
    min_error_loc.append(i[0])
epsilon = epsilon_valid[min_error_loc[0]]
layer_number = layer_number_valid[min_error_loc[1]]
layer_dimension = layer_dimension_valid[min_error_loc[2]]
from sklearn.metrics import classification_report

y_true = y_try
print(layer_number)
model = modelbuiler(layer_number,layer_dimension, itera)
y_prob = guess(model, x_try)
print(max(y_prob[0]))
result = np.argmax(y_prob,0)
print(classification_report(y_try, result))
print(result)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


[[[4831.]]]
3
nan
              precision    recall  f1-score   support

           0       0.03      1.00      0.07       169
           1       0.00      0.00      0.00       160
           2       0.00      0.00      0.00       229
           3       0.00      0.00      0.00       287
           4       0.00      0.00      0.00       196
           5       0.00      0.00      0.00       202
           6       0.00      0.00      0.00       113
           7       0.00      0.00      0.00       236
           8       0.00      0.00      0.00       139
           9       0.00      0.00      0.00       110
          10       0.00      0.00      0.00        92
          11       0.00      0.00      0.00       102
          12       0.00      0.00      0.00       106
          13       0.00      0.00      0.00       129
          14       0.00      0.00      0.00       163
          15       0.00      0.00      0.00       138
          16       0.00      0.00      0.00       255
         

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
